In [1]:
!pip install umap-learn
!pip install scipy
!pip install scikit-learn
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install seaborn
!pip install missingno
!pip install joblib
!pip install scipy

In [2]:
!pip install --upgrade numba umap-learn
!pip install numpy==2.2

# Imports para Preprocesamiento

In [3]:
import pandas as pd
from pathlib import Path
root = Path().resolve()
out = Path(root/'data/')  # Ajusta la ruta según tu caso

# Cargar los DataFrames desde Parquet
df_complete = pd.read_parquet(out / "processed/playlist_tracks_complete.parquet")
df_imputed  = pd.read_parquet(out / "processed/playlist_tracks_imputed.parquet")

# Ahora puedes trabajar con ellos como cualquier DataFrame de pandas
print(df_complete.head())
print(df_imputed.head())

       pid    name  pos       artist_name  \
0  1000000   Party    0         AronChupa   
1  1000000   Party    1         AronChupa   
2  1000000   Party    4             Lorde   
3  1000016  school    0            Alesso   
4  1000016  school    3  American Authors   

                              track_uri  \
0  spotify:track:66U0ASk1VHZsqIkpMjKX3B   
1  spotify:track:5MhsZlmKJG6X5kTHkdwC4B   
2  spotify:track:3G6hD9B2ZHOsgf4WfNu7X1   
3  spotify:track:6WQLkih8nE0JdUCEyLaGnQ   
4  spotify:track:5j9iuo3tMmQIfnEEQOOjxh   

                              artist_uri            track_name  \
0  spotify:artist:5vCOdeiQt9LyzdI87kt5Sh          Little Swing   
1  spotify:artist:5vCOdeiQt9LyzdI87kt5Sh      I'm an Albatraoz   
2  spotify:artist:163tK9Wjr9P9DmM0AVK7lm                  Team   
3  spotify:artist:4AVFqumd2ogHFlRbKIjp1t  Heroes (we could be)   
4  spotify:artist:0MlOPi3zIDMVrfA9R04Fe3   Best Day Of My Life   

                              album_uri  duration_ms        album_name  .

In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, KBinsDiscretizer
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
import joblib
import warnings

# Opcional para Winsorizing (manejo de outliers)
from scipy.stats.mstats import winsorize

# --- Importación Segura de UMAP ---
# Intentamos importar UMAP, si falla por cualquier razón,
# el resto del código no se detendrá.
try:
    import umap.umap_ as umap
    UMAP_AVAILABLE = True
    print("UMAP importado exitosamente.")
except ImportError as e:
    print(f"Advertencia: No se pudo importar UMAP. Error: {e}")
    print("La sección de reducción de dimensionalidad con UMAP será omitida.")
    UMAP_AVAILABLE = False


warnings.filterwarnings('ignore', category=FutureWarning)

# --- DataFrame de partida ---
# Asegúrate de que df_imputed esté cargado de una celda anterior
# y crea una copia para no modificar el original.

#cambiando a complete para solo tener los datos completos
df_processed = df_complete.copy()

print("\nDataFrame inicial para preprocesamiento:")
print(f"Shape: {df_processed.shape}")
df_processed.head()

# ... (El resto del código de preprocesamiento que te proporcioné iría aquí)
# El paso de UMAP al final del script solo se ejecutará si UMAP_AVAILABLE es True.

UMAP importado exitosamente.

DataFrame inicial para preprocesamiento:
Shape: (124096, 123)


,pid,name,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name,...,timbre_bright,timbre_dark,tonal_atonal_value,tonal_atonal_prob,tonal_atonal_atonal,tonal_atonal_tonal,voice_instrumental_value,voice_instrumental_prob,voice_instrumental_instrumental,voice_instrumental_voice
0,1000000,Party,0,AronChupa,spotify:track:66U0ASk1VHZsqIkpMjKX3B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,Little Swing,spotify:album:4S5MLjwRSi0NJ5nikflYnZ,163809,Little Swing,...,0.000930,0.999070,atonal,0.990423,0.990423,0.009577,voice,0.917732,0.082268,0.917732
1,1000000,Party,1,AronChupa,spotify:track:5MhsZlmKJG6X5kTHkdwC4B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,I'm an Albatraoz,spotify:album:1qHVYbxQ6IS8YRviorKDJI,166848,I'm an Albatraoz,...,0.597991,0.402009,atonal,0.913942,0.913942,0.086058,instrumental,0.751203,0.751203,0.248797
2,1000000,Party,4,Lorde,spotify:track:3G6hD9B2ZHOsgf4WfNu7X1,spotify:artist:163tK9Wjr9P9DmM0AVK7lm,Team,spotify:album:0rmhjUgoVa17LZuS8xWQ3v,193058,Pure Heroine,...,0.208725,0.791275,atonal,0.986704,0.986704,0.013296,voice,0.988464,0.011536,0.988464
3,1000016,school,0,Alesso,spotify:track:6WQLkih8nE0JdUCEyLaGnQ,spotify:artist:4AVFqumd2ogHFlRbKIjp1t,Heroes (we could be),spotify:album:3yqh9ozOA2cAgPUJaveAuG,210012,Forever,...,0.752845,0.247155,atonal,0.915288,0.915288,0.084712,voice,0.882508,0.117492,0.882508
4,1000016,school,3,American Authors,spotify:track:5j9iuo3tMmQIfnEEQOOjxh,spotify:artist:0MlOPi3zIDMVrfA9R04Fe3,Best Day Of My Life,spotify:album:2AAVQqcejMEgNpdg2raPYE,194240,"Oh, What A Life",...,0.966806,0.033194,tonal,0.823357,0.176643,0.823357,voice,0.595400,0.404600,0.595400


## 3.1 Eliminar duplicados de tracks

In [5]:
# Seleccionamos las columnas relevantes de las características de audio y el URI
track_feature_cols = ['track_id'] + df_processed.select_dtypes(include=np.number).columns.tolist()
tracks_df_unique = df_processed[track_feature_cols].drop_duplicates(subset=['track_id']).set_index('track_id')

print(f"DataFrame de tracks únicos (sin duplicados por track_id):")
print(f"Shape original de tracks con features: {df_processed.shape[0]}")
print(f"Shape de tracks únicos: {tracks_df_unique.shape[0]}")
tracks_df_unique.head()

DataFrame de tracks únicos (sin duplicados por track_id):
Shape original de tracks con features: 124096
Shape de tracks únicos: 31796


,pid,pos,duration_ms,bpm,energy,danceability_ll,loudness,danceability_prob,danceability_danceable,danceability_not_danceable,...,moods_mirex_Cluster5,timbre_prob,timbre_bright,timbre_dark,tonal_atonal_prob,tonal_atonal_atonal,tonal_atonal_tonal,voice_instrumental_prob,voice_instrumental_instrumental,voice_instrumental_voice
track_id,,,,,,,,,,,,,,,,,,,,,
66U0ASk1VHZsqIkpMjKX3B,1000000,0,163809,125.898476,4.910242,1.067135,0.816509,0.941562,0.941562,0.058438,...,0.224816,0.999070,0.000930,0.999070,0.990423,0.990423,0.009577,0.917732,0.082268,0.917732
5MhsZlmKJG6X5kTHkdwC4B,1000000,1,166848,128.037704,4.296187,0.937844,0.785428,0.507221,0.492779,0.507221,...,0.106123,0.597991,0.597991,0.402009,0.913942,0.913942,0.086058,0.751203,0.751203,0.248797
3G6hD9B2ZHOsgf4WfNu7X1,1000000,4,193058,99.882507,4.392409,1.130975,0.920415,0.999133,0.999133,0.000867,...,0.619994,0.791275,0.208725,0.791275,0.986704,0.986704,0.013296,0.988464,0.011536,0.988464
6WQLkih8nE0JdUCEyLaGnQ,1000016,0,210012,125.980873,2.897875,1.058157,0.840827,0.999994,0.999994,0.000006,...,0.269019,0.752845,0.752845,0.247155,0.915288,0.915288,0.084712,0.882508,0.117492,0.882508
5j9iuo3tMmQIfnEEQOOjxh,1000016,3,194240,100.032249,2.978403,1.249340,0.841262,0.906955,0.906955,0.093045,...,0.128178,0.966806,0.966806,0.033194,0.823357,0.176643,0.823357,0.595400,0.404600,0.595400


Nota: Seguiremos trabajando con df_processed para las tareas que involucran playlists, pero tracks_df_unique es el input correcto para un futuro clustering a nivel de canción.

## 3.2 Tratamiento de NA Categóricos
 <!-- Ya no es necesario porque hemos eliminado los atributos sin valores, el único atributo que queda es nombre, así que le imputaremos el nombre **sin nombre** -->

In [6]:
# --- Configurar pandas para mostrar todas las columnas ---
# Al poner None, le decimos a pandas que no hay límite en el número de columnas a mostrar.
pd.set_option('display.max_columns', None)

# --- Mostrar las 10 primeras filas del DataFrame procesado ---
# El método .head(10) selecciona las primeras 10 filas.
print("Mostrando las 10 primeras filas con todas las columnas visibles:")
df_processed.head(10)

Mostrando las 10 primeras filas con todas las columnas visibles:


,pid,name,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name,track_id,mbid,bpm,energy,danceability_ll,loudness,danceability_value,danceability_prob,danceability_danceable,danceability_not_danceable,gender_value,gender_prob,gender_female,gender_male,genre_dortmund_value,genre_dortmund_prob,genre_dortmund_alternative,genre_dortmund_blues,genre_dortmund_electronic,genre_dortmund_folkcountry,genre_dortmund_funksoulrnb,genre_dortmund_jazz,genre_dortmund_pop,genre_dortmund_raphiphop,genre_dortmund_rock,genre_electronic_value,genre_electronic_prob,genre_electronic_ambient,genre_electronic_dnb,genre_electronic_house,genre_electronic_techno,genre_electronic_trance,genre_rosamerica_value,genre_rosamerica_prob,genre_rosamerica_cla,genre_rosamerica_dan,genre_rosamerica_hip,genre_rosamerica_jaz,genre_rosamerica_pop,genre_rosamerica_rhy,genre_rosamerica_roc,genre_rosamerica_spe,genre_tzanetakis_value,genre_tzanetakis_prob,genre_tzanetakis_blu,genre_tzanetakis_cla,genre_tzanetakis_cou,genre_tzanetakis_dis,genre_tzanetakis_hip,genre_tzanetakis_jaz,genre_tzanetakis_met,genre_tzanetakis_pop,genre_tzanetakis_reg,genre_tzanetakis_roc,ismir04_rhythm_value,ismir04_rhythm_prob,ismir04_rhythm_ChaChaCha,ismir04_rhythm_Jive,ismir04_rhythm_Quickstep,ismir04_rhythm_Rumba-American,ismir04_rhythm_Rumba-International,ismir04_rhythm_Rumba-Misc,ismir04_rhythm_Samba,ismir04_rhythm_Tango,ismir04_rhythm_VienneseWaltz,ismir04_rhythm_Waltz,mood_acoustic_value,mood_acoustic_prob,mood_acoustic_acoustic,mood_acoustic_not_acoustic,mood_aggressive_value,mood_aggressive_prob,mood_aggressive_aggressive,mood_aggressive_not_aggressive,mood_electronic_value,mood_electronic_prob,mood_electronic_electronic,mood_electronic_not_electronic,mood_happy_value,mood_happy_prob,mood_happy_happy,mood_happy_not_happy,mood_party_value,mood_party_prob,mood_party_not_party,mood_party_party,mood_relaxed_value,mood_relaxed_prob,mood_relaxed_not_relaxed,mood_relaxed_relaxed,mood_sad_value,mood_sad_prob,mood_sad_not_sad,mood_sad_sad,moods_mirex_value,moods_mirex_prob,moods_mirex_Cluster1,moods_mirex_Cluster2,moods_mirex_Cluster3,moods_mirex_Cluster4,moods_mirex_Cluster5,timbre_value,timbre_prob,timbre_bright,timbre_dark,tonal_atonal_value,tonal_atonal_prob,tonal_atonal_atonal,tonal_atonal_tonal,voice_instrumental_value,voice_instrumental_prob,voice_instrumental_instrumental,voice_instrumental_voice
0,1000000,Party,0,AronChupa,spotify:track:66U0ASk1VHZsqIkpMjKX3B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,Little Swing,spotify:album:4S5MLjwRSi0NJ5nikflYnZ,163809,Little Swing,66U0ASk1VHZsqIkpMjKX3B,302944eb-7cb7-4045-b99c-c440c362718f,125.898476,4.910242,1.067135,0.816509,danceable,0.941562,0.941562,0.058438,female,0.838504,0.838504,0.161496,electronic,0.958349,1.293055e-02,4.026770e-03,0.958349,0.004129,7.821612e-04,0.007588,0.001415,0.007156,3.623097e-03,ambient,0.633230,0.633230,0.022131,0.248523,0.030331,0.065786,rhy,0.319759,0.017242,0.157987,0.295804,0.031568,0.147856,0.319759,0.008005,0.021779,pop,0.201998,0.057697,0.080738,0.080722,0.100906,0.080744,0.134602,0.080795,0.201998,0.101015,0.080782,Tango,0.773513,0.057953,0.015954,0.004549,0.008992,0.022028,0.010735,0.035496,0.773513,0.067657,0.003123,not_acoustic,0.927582,0.072418,0.927582,not_aggressive,0.660399,3.396006e-01,0.660399,electronic,0.905014,0.905014,0.094986,not_happy,0.967353,0.032647,0.967353,party,0.660808,0.339192,0.660808,relaxed,0.927280,0.072720,0.927280,not_sad,0.527017,0.527017,0.472983,Cluster4,0.253382,0.143356,0.238793,0.139652,0.253382,0.224816,dark,0.999070,0.000930,0.999070,atonal,0.990423,0.990423,0.009577,voice,0.917732,0.082268,0.917732
1,1000000,Party,1,AronChupa,spotify:track:5MhsZlmKJG6X5kTHkdwC4B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,I'm an Albatraoz,spotify:album:1qHVYbxQ6IS8YRviorKDJI,166848,I'm an Albatraoz,5MhsZlmKJG6X5kTHkdwC4B,466973b8-6f13-4cdd-aed5-1c8cd6b26fb4,128.037704,4.296187,0.937844,0.785428,not_danceable,0.507221,0.492779,0.507221,male,0.590108,0.409892,0.590108,elec

In [7]:
# Identificar columnas categóricas (tipo 'object')
categorical_cols = df_processed.select_dtypes(include=['object', 'category']).columns.tolist()

# # El track_id no debe ser imputado, lo removemos si está en la lista
# if 'track_id' in categorical_cols:
#     categorical_cols.remove('track_id')

# Crear el imputer
# imputer_cat = SimpleImputer(strategy='constant', fill_value='NA')

# # Aplicar el imputer
# df_processed[categorical_cols] = imputer_cat.fit_transform(df_processed[categorical_cols])

df_processed['name'] = df_processed['name'].fillna('sin nombre')

print("Porcentaje de nulos después de imputar categóricas:")
print(df_processed[categorical_cols].isnull().sum() / len(df_processed) * 100)
df_processed.head()

Porcentaje de nulos después de imputar categóricas:
name                        0.0
artist_name                 0.0
track_uri                   0.0
artist_uri                  0.0
track_name                  0.0
album_uri                   0.0
album_name                  0.0
track_id                    0.0
mbid                        0.0
danceability_value          0.0
gender_value                0.0
genre_dortmund_value        0.0
genre_electronic_value      0.0
genre_rosamerica_value      0.0
genre_tzanetakis_value      0.0
ismir04_rhythm_value        0.0
mood_acoustic_value         0.0
mood_aggressive_value       0.0
mood_electronic_value       0.0
mood_happy_value            0.0
mood_party_value            0.0
mood_relaxed_value          0.0
mood_sad_value              0.0
moods_mirex_value           0.0
timbre_value                0.0
tonal_atonal_value          0.0
voice_instrumental_value    0.0
dtype: float64


,pid,name,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name,track_id,mbid,bpm,energy,danceability_ll,loudness,danceability_value,danceability_prob,danceability_danceable,danceability_not_danceable,gender_value,gender_prob,gender_female,gender_male,genre_dortmund_value,genre_dortmund_prob,genre_dortmund_alternative,genre_dortmund_blues,genre_dortmund_electronic,genre_dortmund_folkcountry,genre_dortmund_funksoulrnb,genre_dortmund_jazz,genre_dortmund_pop,genre_dortmund_raphiphop,genre_dortmund_rock,genre_electronic_value,genre_electronic_prob,genre_electronic_ambient,genre_electronic_dnb,genre_electronic_house,genre_electronic_techno,genre_electronic_trance,genre_rosamerica_value,genre_rosamerica_prob,genre_rosamerica_cla,genre_rosamerica_dan,genre_rosamerica_hip,genre_rosamerica_jaz,genre_rosamerica_pop,genre_rosamerica_rhy,genre_rosamerica_roc,genre_rosamerica_spe,genre_tzanetakis_value,genre_tzanetakis_prob,genre_tzanetakis_blu,genre_tzanetakis_cla,genre_tzanetakis_cou,genre_tzanetakis_dis,genre_tzanetakis_hip,genre_tzanetakis_jaz,genre_tzanetakis_met,genre_tzanetakis_pop,genre_tzanetakis_reg,genre_tzanetakis_roc,ismir04_rhythm_value,ismir04_rhythm_prob,ismir04_rhythm_ChaChaCha,ismir04_rhythm_Jive,ismir04_rhythm_Quickstep,ismir04_rhythm_Rumba-American,ismir04_rhythm_Rumba-International,ismir04_rhythm_Rumba-Misc,ismir04_rhythm_Samba,ismir04_rhythm_Tango,ismir04_rhythm_VienneseWaltz,ismir04_rhythm_Waltz,mood_acoustic_value,mood_acoustic_prob,mood_acoustic_acoustic,mood_acoustic_not_acoustic,mood_aggressive_value,mood_aggressive_prob,mood_aggressive_aggressive,mood_aggressive_not_aggressive,mood_electronic_value,mood_electronic_prob,mood_electronic_electronic,mood_electronic_not_electronic,mood_happy_value,mood_happy_prob,mood_happy_happy,mood_happy_not_happy,mood_party_value,mood_party_prob,mood_party_not_party,mood_party_party,mood_relaxed_value,mood_relaxed_prob,mood_relaxed_not_relaxed,mood_relaxed_relaxed,mood_sad_value,mood_sad_prob,mood_sad_not_sad,mood_sad_sad,moods_mirex_value,moods_mirex_prob,moods_mirex_Cluster1,moods_mirex_Cluster2,moods_mirex_Cluster3,moods_mirex_Cluster4,moods_mirex_Cluster5,timbre_value,timbre_prob,timbre_bright,timbre_dark,tonal_atonal_value,tonal_atonal_prob,tonal_atonal_atonal,tonal_atonal_tonal,voice_instrumental_value,voice_instrumental_prob,voice_instrumental_instrumental,voice_instrumental_voice
0,1000000,Party,0,AronChupa,spotify:track:66U0ASk1VHZsqIkpMjKX3B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,Little Swing,spotify:album:4S5MLjwRSi0NJ5nikflYnZ,163809,Little Swing,66U0ASk1VHZsqIkpMjKX3B,302944eb-7cb7-4045-b99c-c440c362718f,125.898476,4.910242,1.067135,0.816509,danceable,0.941562,0.941562,0.058438,female,0.838504,0.838504,0.161496,electronic,0.958349,0.012931,0.004027,0.958349,0.004129,0.000782,0.007588,0.001415,0.007156,0.003623,ambient,0.633230,0.633230,0.022131,0.248523,0.030331,0.065786,rhy,0.319759,0.017242,0.157987,0.295804,0.031568,0.147856,0.319759,0.008005,0.021779,pop,0.201998,0.057697,0.080738,0.080722,0.100906,0.080744,0.134602,0.080795,0.201998,0.101015,0.080782,Tango,0.773513,0.057953,0.015954,0.004549,0.008992,0.022028,0.010735,0.035496,0.773513,0.067657,0.003123,not_acoustic,0.927582,0.072418,0.927582,not_aggressive,0.660399,3.396006e-01,0.660399,electronic,0.905014,0.905014,0.094986,not_happy,0.967353,0.032647,0.967353,party,0.660808,0.339192,0.660808,relaxed,0.927280,0.072720,0.927280,not_sad,0.527017,0.527017,0.472983,Cluster4,0.253382,0.143356,0.238793,0.139652,0.253382,0.224816,dark,0.999070,0.000930,0.999070,atonal,0.990423,0.990423,0.009577,voice,0.917732,0.082268,0.917732
1,1000000,Party,1,AronChupa,spotify:track:5MhsZlmKJG6X5kTHkdwC4B,spotify:artist:5vCOdeiQt9LyzdI87kt5Sh,I'm an Albatraoz,spotify:album:1qHVYbxQ6IS8YRviorKDJI,166848,I'm an Albatraoz,5MhsZlmKJG6X5kTHkdwC4B,466973b8-6f13-4cdd-aed5-1c8cd6b26fb4,128.037704,4.296187,0.937844,0.785428,not_danceable,0.507221,0.492779,0.507221,male,0.590108,0.409892,0.590108,electronic,0.991147,

##3.3 Eliminando columnas irrelevantes

In [8]:
columnas = set(df_processed.columns)

In [9]:
columnas

{'album_name',
 'album_uri',
 'artist_name',
 'artist_uri',
 'bpm',
 'danceability_danceable',
 'danceability_ll',
 'danceability_not_danceable',
 'danceability_prob',
 'danceability_value',
 'duration_ms',
 'energy',
 'gender_female',
 'gender_male',
 'gender_prob',
 'gender_value',
 'genre_dortmund_alternative',
 'genre_dortmund_blues',
 'genre_dortmund_electronic',
 'genre_dortmund_folkcountry',
 'genre_dortmund_funksoulrnb',
 'genre_dortmund_jazz',
 'genre_dortmund_pop',
 'genre_dortmund_prob',
 'genre_dortmund_raphiphop',
 'genre_dortmund_rock',
 'genre_dortmund_value',
 'genre_electronic_ambient',
 'genre_electronic_dnb',
 'genre_electronic_house',
 'genre_electronic_prob',
 'genre_electronic_techno',
 'genre_electronic_trance',
 'genre_electronic_value',
 'genre_rosamerica_cla',
 'genre_rosamerica_dan',
 'genre_rosamerica_hip',
 'genre_rosamerica_jaz',
 'genre_rosamerica_pop',
 'genre_rosamerica_prob',
 'genre_rosamerica_rhy',
 'genre_rosamerica_roc',
 'genre_rosamerica_spe',
 '

Para cada canción decidimos priorizar la fidelidad de los datos eliminando solo las columnas que consideramos irrelevantes.

Para entenderlo vamos a ver solo la categoría **gender**, en el json se vería así
```json
 "gender": {
        "all": {
          "female": 0.838503777981,
          "male": 0.161496207118
        },
        "probability": 0.838503777981,
        "value": "female",
        }
```

La categoría `gender_value` nos parece irrelevante porque solo nos mostraría cual de los dos valores que género podría tomar representa la mayor probabilidad

La categoría `gender_probability` nos parece redundante porque replica la probabilidad de gender_value, es decir la probabilidad entre los valores de gender más alta

con `gender_male` y `gender_female` teníamos dos opciones; guardar las dos o guardar solo una. En este caso decidimos entonces escoger solo uno de los valores, en este caso `gender_female` ya que la 1-gender_female es gender_male, por lo que están relacionados y sería un poco redundante añadirlo

En otros casos en donde existen más de un subitem, decidimos guardar todas las probabilidades que describen a los subitems, eliminando entonces el item_value y el item_prob. Esta sería una lista de los valores que vamos a guardar para cada canción.

## 📀 Columnas por canción (track)

| Bloque | Columnas resultantes (nombre sugerido) |
|--------|----------------------------------------|
| **Identidad / metadatos** | `track_id`, `duration_ms`, `album_name`, `artist_name`, `track_name`, `name`,`pid`,
| **Audio numérico crudo** | `bpm`, `energy`, `danceability_ll`, `loudness` |
| **Probabilidades high‑level** |  |
| · **Danceability** | `prob_danceable` |
| · **Gender** | `prob_female` |
| · **genre_dortmund** (9) | `p_alt`, `p_blues`, `p_electronic`, `p_folkcountry`, `p_funksoulrnb`, `p_jazz`, `p_pop`, `p_raphiphop`, `p_rock` |
| · **genre_electronic** (5) | `p_ambient`, `p_dnb`, `p_house`, `p_techno`, `p_trance` |
| · **genre_rosamerica** (8) | `p_cla_ros`, `p_dan`, `p_hip_ros`, `p_jaz_ros`, `p_pop_ros`, `p_rhy`, `p_roc_ros`, `p_spe` |
| · **genre_tzanetakis** (10) | `p_blu_tza`, `p_cla_tza`, `p_cou`, `p_dis`, `p_hip_tza`, `p_jaz_tza`, `p_met`, `p_pop_tza`, `p_reg`, `p_roc_tza` |
| · **ismir04_rhythm** (10) | `p_Chacha`, `p_Jive`, `p_Quickstep`, `p_RumbaAm`, `p_RumbaInt`, `p_RumbaMisc`, `p_Samba`, `p_Tango`, `p_Viennese`, `p_Waltz` |
| · **mood_acoustic** | `prob_acoustic` |
| · **mood_aggressive** | `prob_aggressive` |
| · **mood_electronic** | `prob_mood_electronic` |
| · **mood_happy / party / relaxed / sad** | `prob_happy`, `prob_party`, `prob_relaxed`, `prob_sad` |
| · **moods_mirex** (5) | `p_cluster1`, `p_cluster2`, `p_cluster3`, `p_cluster4`, `p_cluster5` |
| · **timbre** | `prob_bright` |
| · **tonal_atonal** | `prob_atonal` |
| · **voice_instrumental** | `prob_voice` |

> **Total aproximado:** ~ 70 columnas por pista (4 métricas audio + ~ 58 probabilidades + metadatos).


In [10]:
df_processed = df_processed = df_processed.drop(
    columns=[
        # ─── metadatos innecesarios ─────────────────────────────
          "pos", "mbid", "artist_uri", "album_uri", "track_uri",

        # ─── danceability ───────────────────────────────────────
        "danceability_not_danceable", "danceability_prob", "danceability_value",

        # ─── gender ─────────────────────────────────────────────
        "gender_male", "gender_prob", "gender_value",

        # ─── géneros ────────────────────────────────────────────
        "genre_dortmund_prob", "genre_dortmund_value",
        "genre_electronic_prob", "genre_electronic_value",
        "genre_rosamerica_prob", "genre_rosamerica_value",
        "genre_tzanetakis_prob", "genre_tzanetakis_value",

        # ─── ritmo ISMIR‑04 ─────────────────────────────────────
        "ismir04_rhythm_prob", "ismir04_rhythm_value",

        # ─── moods ──────────────────────────────────────────────
        "mood_acoustic_not_acoustic", "mood_acoustic_prob", "mood_acoustic_value",
        "mood_aggressive_not_aggressive", "mood_aggressive_prob", "mood_aggressive_value",
        "mood_electronic_not_electronic", "mood_electronic_prob", "mood_electronic_value",
        "mood_happy_not_happy", "mood_happy_prob", "mood_happy_value",
        "mood_party_not_party", "mood_party_prob", "mood_party_value",
        "mood_relaxed_not_relaxed", "mood_relaxed_prob", "mood_relaxed_value",
        "mood_sad_not_sad", "mood_sad_prob", "mood_sad_value",

        # ─── moods MIREX ────────────────────────────────────────
        "moods_mirex_prob", "moods_mirex_value",

        # ─── timbre & tonalidad ─────────────────────────────────
        "timbre_dark", "timbre_prob", "timbre_value",
        "tonal_atonal_tonal", "tonal_atonal_prob", "tonal_atonal_value",

        # ─── voz‑instrumental ───────────────────────────────────
        "voice_instrumental_instrumental", "voice_instrumental_prob", "voice_instrumental_value",
    ]
)

print("\nDataFrame after dropping specified columns and genre columns:")
print(f"Shape: {df_processed.shape}")
print("Remaining columns:")
print(df_processed.columns.tolist())
df_processed.head()


DataFrame after dropping specified columns and genre columns:
Shape: (124096, 70)
Remaining columns:
['pid', 'name', 'artist_name', 'track_name', 'duration_ms', 'album_name', 'track_id', 'bpm', 'energy', 'danceability_ll', 'loudness', 'danceability_danceable', 'gender_female', 'genre_dortmund_alternative', 'genre_dortmund_blues', 'genre_dortmund_electronic', 'genre_dortmund_folkcountry', 'genre_dortmund_funksoulrnb', 'genre_dortmund_jazz', 'genre_dortmund_pop', 'genre_dortmund_raphiphop', 'genre_dortmund_rock', 'genre_electronic_ambient', 'genre_electronic_dnb', 'genre_electronic_house', 'genre_electronic_techno', 'genre_electronic_trance', 'genre_rosamerica_cla', 'genre_rosamerica_dan', 'genre_rosamerica_hip', 'genre_rosamerica_jaz', 'genre_rosamerica_pop', 'genre_rosamerica_rhy', 'genre_rosamerica_roc', 'genre_rosamerica_spe', 'genre_tzanetakis_blu', 'genre_tzanetakis_cla', 'genre_tzanetakis_cou', 'genre_tzanetakis_dis', 'genre_tzanetakis_hip', 'genre_tzanetakis_jaz', 'genre_tzaneta

,pid,name,artist_name,track_name,duration_ms,album_name,track_id,bpm,energy,danceability_ll,loudness,danceability_danceable,gender_female,genre_dortmund_alternative,genre_dortmund_blues,genre_dortmund_electronic,genre_dortmund_folkcountry,genre_dortmund_funksoulrnb,genre_dortmund_jazz,genre_dortmund_pop,genre_dortmund_raphiphop,genre_dortmund_rock,genre_electronic_ambient,genre_electronic_dnb,genre_electronic_house,genre_electronic_techno,genre_electronic_trance,genre_rosamerica_cla,genre_rosamerica_dan,genre_rosamerica_hip,genre_rosamerica_jaz,genre_rosamerica_pop,genre_rosamerica_rhy,genre_rosamerica_roc,genre_rosamerica_spe,genre_tzanetakis_blu,genre_tzanetakis_cla,genre_tzanetakis_cou,genre_tzanetakis_dis,genre_tzanetakis_hip,genre_tzanetakis_jaz,genre_tzanetakis_met,genre_tzanetakis_pop,genre_tzanetakis_reg,genre_tzanetakis_roc,ismir04_rhythm_ChaChaCha,ismir04_rhythm_Jive,ismir04_rhythm_Quickstep,ismir04_rhythm_Rumba-American,ismir04_rhythm_Rumba-International,ismir04_rhythm_Rumba-Misc,ismir04_rhythm_Samba,ismir04_rhythm_Tango,ismir04_rhythm_VienneseWaltz,ismir04_rhythm_Waltz,mood_acoustic_acoustic,mood_aggressive_aggressive,mood_electronic_electronic,mood_happy_happy,mood_party_party,mood_relaxed_relaxed,mood_sad_sad,moods_mirex_Cluster1,moods_mirex_Cluster2,moods_mirex_Cluster3,moods_mirex_Cluster4,moods_mirex_Cluster5,timbre_bright,tonal_atonal_atonal,voice_instrumental_voice
0,1000000,Party,AronChupa,Little Swing,163809,Little Swing,66U0ASk1VHZsqIkpMjKX3B,125.898476,4.910242,1.067135,0.816509,0.941562,0.838504,0.012931,0.004027,0.958349,0.004129,0.000782,0.007588,0.001415,0.007156,0.003623,0.633230,0.022131,0.248523,0.030331,0.065786,0.017242,0.157987,0.295804,0.031568,0.147856,0.319759,0.008005,0.021779,0.057697,0.080738,0.080722,0.100906,0.080744,0.134602,0.080795,0.201998,0.101015,0.080782,0.057953,0.015954,0.004549,0.008992,0.022028,0.010735,0.035496,0.773513,0.067657,0.003123,0.072418,3.396006e-01,0.905014,0.032647,0.660808,0.927280,0.472983,0.143356,0.238793,0.139652,0.253382,0.224816,0.000930,0.990423,0.917732
1,1000000,Party,AronChupa,I'm an Albatraoz,166848,I'm an Albatraoz,5MhsZlmKJG6X5kTHkdwC4B,128.037704,4.296187,0.937844,0.785428,0.492779,0.409892,0.001149,0.001203,0.991147,0.000679,0.000228,0.004117,0.000108,0.000572,0.000797,0.145308,0.018037,0.759690,0.035563,0.041402,0.014970,0.198387,0.431846,0.089087,0.050573,0.197856,0.010846,0.006436,0.039472,0.078931,0.063129,0.052604,0.045090,0.157818,0.063133,0.105224,0.078920,0.315679,0.069085,0.027921,0.003872,0.008992,0.006563,0.002090,0.018909,0.851566,0.008959,0.002044,0.000006,3.000001e-14,0.917834,0.013071,0.000836,0.808827,0.307780,0.109993,0.200788,0.248826,0.334271,0.106123,0.597991,0.913942,0.248797
2,1000000,Party,Lorde,Team,193058,Pure Heroine,3G6hD9B2ZHOsgf4WfNu7X1,99.882507,4.392409,1.130975,0.920415,0.999133,0.721461,0.027483,0.015242,0.860394,0.042829,0.002202,0.015942,0.013363,0.007874,0.014671,0.297171,0.108914,0.426002,0.048630,0.119284,0.007912,0.099371,0.143291,0.017134,0.226016,0.473656,0.018283,0.014337,0.062067,0.034475,0.103434,0.062097,0.155272,0.310667,0.044347,0.062079,0.062082,0.103480,0.883623,0.009685,0.003651,0.009970,0.032320,0.004501,0.016806,0.007797,0.029669,0.001980,0.015300,7.174270e-01,0.715190,0.054321,0.774370,0.420903,0.234746,0.077223,0.044405,0.227747,0.030632,0.619994,0.208725,0.986704,0.988464
3,1000016,school,Alesso,Heroes (we could be),210012,Forever,6WQLkih8nE0JdUCEyLaGnQ,125.980873,2.897875,1.058157,0.840827,0.999994,0.693479,0.041034,0.009111,0.914271,0.018788,0.000727,0.003867,0.003554,0.000694,0.007955,0.349274,0.052637,0.193606,0.031518,0.372966,0.001067,0.084875,0.063193,0.027307,0.759889,0.042204,0.020546,0.000921,0.062661,0.034817,0.104476,0.078432,0.156924,0.313975,0.044817,0.062736,0.062736,0.078426,0.256939,0.118988,0.028248,0.076212,0.070221,0.037244,0.151571,0.125094,0.113810,0.021673,0.006676,3.000001e-14,0.904654,0.654065,0.000062,0.808817,0.431928,0.114918,0.255265,0.191129,0.169670,0.269019,0.75

importante tener en cuenta no agregar metadatos en el procesamiento de clusters
"album_name", "artist_name", "track_name", "name","trackid"

# 3.5 Tratamiento de Outliers (Winsorizing)
Antes de escalar, es buena idea tratar los outliers identificados en el EDA para que no influyan desproporcionadamente en el escalado. Usaremos Winsorizing, que recorta los valores extremos a un percentil determinado.

In [11]:
# --- 3.3 Tratamiento de Outliers (Opcional pero recomendado) ---

# Identificar columnas numéricas (excluyendo IDs y flags)
numerical_cols = df_processed.select_dtypes(include=np.number).columns.tolist()
# Excluir identificadores o columnas que no deban ser tratadas
cols_to_exclude = ['pid', 'pos', 'duration_ms', 'was_imputed']
numerical_cols = [col for col in numerical_cols if col not in cols_to_exclude]

# Aplicar Winsorizing al 1% y 99%
for col in numerical_cols:
    df_processed[col] = winsorize(df_processed[col], limits=[0.01, 0.01])

print("Valores de 'energy' después de Winsorizing:")
df_processed['energy'].describe()

Valores de 'energy' después de Winsorizing:


/usr/local/lib/python3.11/dist-packages/numpy/lib/_function_base_impl.py:4842: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  arr.partition(


,energy
count,124096.000000
mean,3.996207
std,1.354402
min,1.613505
25%,3.064914
50%,3.754053
75%,4.673932
max,9.014983


# 3.4 Normalizar Numéricos (StandardScaler)
Este es un paso fundamental para los algoritmos de clustering como K-Means.

In [12]:
# --- 3.4 Normalizar Numéricos ---

# Usaremos las mismas columnas numéricas de antes
print(f"Columnas a escalar: {numerical_cols}")

# 1. Crear el objeto scaler
scaler = StandardScaler()

# 2. Ajustar y transformar los datos
df_processed[numerical_cols] = scaler.fit_transform(df_processed[numerical_cols])

# 3. Guardar el scaler para uso futuro (por ejemplo, en producción)
joblib.dump(scaler, 'scaler_audio_features.joblib')
print("\nScaler guardado como 'scaler_audio_features.joblib'")

print("\nDataFrame después del escalado (verificar media ~0 y std ~1):")
df_processed[numerical_cols].describe().round(2)

Columnas a escalar: ['bpm', 'energy', 'danceability_ll', 'loudness', 'danceability_danceable', 'gender_female', 'genre_dortmund_alternative', 'genre_dortmund_blues', 'genre_dortmund_electronic', 'genre_dortmund_folkcountry', 'genre_dortmund_funksoulrnb', 'genre_dortmund_jazz', 'genre_dortmund_pop', 'genre_dortmund_raphiphop', 'genre_dortmund_rock', 'genre_electronic_ambient', 'genre_electronic_dnb', 'genre_electronic_house', 'genre_electronic_techno', 'genre_electronic_trance', 'genre_rosamerica_cla', 'genre_rosamerica_dan', 'genre_rosamerica_hip', 'genre_rosamerica_jaz', 'genre_rosamerica_pop', 'genre_rosamerica_rhy', 'genre_rosamerica_roc', 'genre_rosamerica_spe', 'genre_tzanetakis_blu', 'genre_tzanetakis_cla', 'genre_tzanetakis_cou', 'genre_tzanetakis_dis', 'genre_tzanetakis_hip', 'genre_tzanetakis_jaz', 'genre_tzanetakis_met', 'genre_tzanetakis_pop', 'genre_tzanetakis_reg', 'genre_tzanetakis_roc', 'ismir04_rhythm_ChaChaCha', 'ismir04_rhythm_Jive', 'ismir04_rhythm_Quickstep', 'ismir

,bpm,energy,danceability_ll,loudness,danceability_danceable,gender_female,genre_dortmund_alternative,genre_dortmund_blues,genre_dortmund_electronic,genre_dortmund_folkcountry,genre_dortmund_funksoulrnb,genre_dortmund_jazz,genre_dortmund_pop,genre_dortmund_raphiphop,genre_dortmund_rock,genre_electronic_ambient,genre_electronic_dnb,genre_electronic_house,genre_electronic_techno,genre_electronic_trance,genre_rosamerica_cla,genre_rosamerica_dan,genre_rosamerica_hip,genre_rosamerica_jaz,genre_rosamerica_pop,genre_rosamerica_rhy,genre_rosamerica_roc,genre_rosamerica_spe,genre_tzanetakis_blu,genre_tzanetakis_cla,genre_tzanetakis_cou,genre_tzanetakis_dis,genre_tzanetakis_hip,genre_tzanetakis_jaz,genre_tzanetakis_met,genre_tzanetakis_pop,genre_tzanetakis_reg,genre_tzanetakis_roc,ismir04_rhythm_ChaChaCha,ismir04_rhythm_Jive,ismir04_rhythm_Quickstep,ismir04_rhythm_Rumba-American,ismir04_rhythm_Rumba-International,ismir04_rhythm_Rumba-Misc,ismir04_rhythm_Samba,ismir04_rhythm_Tango,ismir04_rhythm_VienneseWaltz,ismir04_rhythm_Waltz,mood_acoustic_acoustic,mood_aggressive_aggressive,mood_electronic_electronic,mood_happy_happy,mood_party_party,mood_relaxed_relaxed,mood_sad_sad,moods_mirex_Cluster1,moods_mirex_Cluster2,moods_mirex_Cluster3,moods_mirex_Cluster4,moods_mirex_Cluster5,timbre_bright,tonal_atonal_atonal,voice_instrumental_voice
count,124096.00,124096.00,124096.00,124096.00,124096.00,124096.00,124096.00,124096.00,124096.00,124096.00,124096.00,124096.00,124096.00,124096.00,124096.00,124096.00,124096.00,124096.00,124096.00,124096.00,124096.00,124096.00,124096.00,124096.00,124096.00,124096.00,124096.00,124096.00,124096.00,124096.00,124096.00,124096.00,124096.00,124096.00,124096.00,124096.00,124096.00,124096.00,124096.00,124096.00,124096.00,124096.00,124096.00,124096.00,124096.00,124096.00,124096.00,124096.00,124096.00,124096.00,124096.00,124096.00,124096.00,124096.00,124096.00,124096.00,124096.00,124096.00,124096.00,124096.00,124096.00,124096.00,124096.00
mean,0.00,-0.00,0.00,-0.00,-0.00,-0.00,-0.00,0.00,-0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.00,0.00,0.00,0.00,-0.00,-0.00,0.00,-0.00,0.00,-0.00,-0.00,-0.00,-0.00,0.00,0.00,-0.00,0.00,0.00,0.00,0.00,-0.00,0.00,-0.00,-0.00,-0.00,0.00,0.00,-0.00,0.00,-0.00,0.00,0.00,-0.00,0.00,0.00,-0.00,-0.00,0.00,0.00,-0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.00
std,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
min,-1.78,-1.76,-1.70,-4.08,-1.81,-1.50,-0.70,-0.53,-3.04,-0.53,-0.58,-0.53,-0.55,-0.23,-0.52,-1.59,-0.94,-1.12,-1.03,-1.23,-0.36,-0.55,-0.63,-0.48,-1.12,-1.01,-0.65,-0.72,-4.03,-2.38,-3.59,-3.21,-3.56,-1.46,-3.16,-2.65,-3.87,-1.94,-1.06,-1.10,-1.24,-0.55,-0.76,-1.08,-1.16,-0.86,-0.88,-0.68,-0.82,-0.66,-1.74,-1.46,-1.36,-1.73,-1.53,-1.50,-1.29,-1.54,-1.58,-1.41,-1.44,-1.55,-2.06
25%,-0.88,-0.69,-0.71,-0.18,-0.90,-1.03,-0.66,-0.52,-0.26,-0.52,-0.58,-0.52,-0.54,-0.23,-0.51,-0.83,-0.61,-0.77,-0.59,-0.86,-0.34,-0.49,-0.59,-0.44,-0.86,-0.81,-0.62,-0.60,0.20,-0.06,-0.44,-0.33,0.32,-0.12,-0.13,-0.14,0.08,-0.38,-0.77,-0.88,-0.91,-0.39,-0.58,-0.80,-0.85,-0.70,-0.73,-0.51,-0.76,-0.64,-0.94,-0.87,-0.97,-0.96,-0.80,-0.66,-0.78,-0.78,-0.75,-0.87,-1.08,-1.06,-0.70
50%,0.00,-0.18,-0.17,0.37,0.56,0.22,-0.48,-0.45,0.53,-0.46,-0.51,-0.42,-0.48,-0.22,-0.45,-0.13,-0.27,-0.31,-0.25,-0.26,-0.29,-0.38,-0.49,-0.34,-0.30,-0.35,-0.51,-0.36,0.24,-0.04,0.49,0.49,0.39,-0.10,-0.10,-0.12,0.12,-0.38,-0.45,-0.27,-0.18,-0.18,-0.26,-0.23,-0.23,-0.31,-0.40,-0.14,-0.51,-0.47,0.09,-0.15,-0.03,0.30,-0.15,-0.24,-0.22,-0.10,-0.08,-0.20,0.18,0.26,0.46
75%,0.73,0.50,0.50,0.64,0.86,0.97,0.27,0.01,0.66,-0.03,0.09,0.10,0.03,-0.12,-0.02,0.70,0.26,0.50,0.21,0.67,-0.16,-0.04,0.05,-0.06,0.69,0.56,0.15,0.17,0.24,-0.04,0.53,0.49,0.39,-0.10,-0.10,-0.12,0.12,0.11,0.78,0.65,0.74,0.08,0.15

In [13]:
df_processed.head()

,pid,name,artist_name,track_name,duration_ms,album_name,track_id,bpm,energy,danceability_ll,loudness,danceability_danceable,gender_female,genre_dortmund_alternative,genre_dortmund_blues,genre_dortmund_electronic,genre_dortmund_folkcountry,genre_dortmund_funksoulrnb,genre_dortmund_jazz,genre_dortmund_pop,genre_dortmund_raphiphop,genre_dortmund_rock,genre_electronic_ambient,genre_electronic_dnb,genre_electronic_house,genre_electronic_techno,genre_electronic_trance,genre_rosamerica_cla,genre_rosamerica_dan,genre_rosamerica_hip,genre_rosamerica_jaz,genre_rosamerica_pop,genre_rosamerica_rhy,genre_rosamerica_roc,genre_rosamerica_spe,genre_tzanetakis_blu,genre_tzanetakis_cla,genre_tzanetakis_cou,genre_tzanetakis_dis,genre_tzanetakis_hip,genre_tzanetakis_jaz,genre_tzanetakis_met,genre_tzanetakis_pop,genre_tzanetakis_reg,genre_tzanetakis_roc,ismir04_rhythm_ChaChaCha,ismir04_rhythm_Jive,ismir04_rhythm_Quickstep,ismir04_rhythm_Rumba-American,ismir04_rhythm_Rumba-International,ismir04_rhythm_Rumba-Misc,ismir04_rhythm_Samba,ismir04_rhythm_Tango,ismir04_rhythm_VienneseWaltz,ismir04_rhythm_Waltz,mood_acoustic_acoustic,mood_aggressive_aggressive,mood_electronic_electronic,mood_happy_happy,mood_party_party,mood_relaxed_relaxed,mood_sad_sad,moods_mirex_Cluster1,moods_mirex_Cluster2,moods_mirex_Cluster3,moods_mirex_Cluster4,moods_mirex_Cluster5,timbre_bright,tonal_atonal_atonal,voice_instrumental_voice
0,1000000,Party,AronChupa,Little Swing,163809,Little Swing,66U0ASk1VHZsqIkpMjKX3B,0.160221,0.674865,-0.575001,-0.010197,0.712000,0.890641,-0.498757,-0.430456,0.519981,-0.481145,-0.433349,-0.127378,-0.436943,-0.045659,-0.443713,0.879014,-0.356821,0.216197,0.166095,-0.988342,-0.187154,0.366962,0.586453,-0.120611,-0.472561,0.371606,-0.618070,1.112369,-0.095358,3.040138,-0.406131,1.546956,-1.530079,-1.287515,2.352964,5.155002,2.426936,-0.335286,-0.888938,-0.840955,-0.875683,-0.428931,-0.389895,-0.568154,-0.426928,3.116291,-0.611418,-0.549471,-0.581131,0.685640,1.028789,-1.346029,0.786925,1.130548,0.525529,0.893153,0.420956,-0.730197,1.004184,-0.387829,-1.441935,1.124915,0.680423
1,1000000,Party,AronChupa,I'm an Albatraoz,166848,I'm an Albatraoz,5MhsZlmKJG6X5kTHkdwC4B,0.245847,0.221486,-1.317574,-0.183550,-0.488870,-0.333640,-0.679784,-0.500463,0.647566,-0.520152,-0.540629,-0.312999,-0.539862,-0.219262,-0.502418,-1.123346,-0.471502,3.101279,0.397027,-1.088660,-0.209497,0.602960,1.148218,0.543977,-0.904793,-0.155714,-0.607289,-0.184139,-1.338528,2.918893,-1.099537,-0.730432,-2.426967,-1.133561,1.148927,1.597147,1.152033,4.106680,-0.854069,-0.646158,-0.930438,-0.428940,-0.652143,-0.990493,-0.774483,3.517509,-0.847052,-0.596331,-0.817869,-0.660169,1.069335,-1.418720,-1.359290,0.761099,-0.247641,0.109339,0.095873,0.124757,1.942266,-0.973718,0.130479,0.917806,-1.320410
2,1000000,Party,Lorde,Team,193058,Pure Heroine,3G6hD9B2ZHOsgf4WfNu7X1,-0.881112,0.292530,-0.208342,0.569353,0.866053,0.556323,-0.275158,-0.152407,0.138928,-0.043543,-0.158424,0.319389,0.504042,-0.026728,-0.214233,-0.500124,2.074336,1.217905,0.973788,-0.768241,-0.278889,0.024549,-0.043327,-0.287384,-0.125290,1.037323,-0.579063,0.483531,0.202774,-0.064239,0.489081,-0.282863,0.344680,-0.119944,-0.131801,-0.142258,0.081347,0.114261,1.697582,-0.942994,-0.948384,-0.415205,-0.215368,-0.872695,-0.818559,-0.819740,-0.763915,-0.599103,-0.767869,2.182934,0.428434,-1.265545,1.156226,-0.448820,-0.589449,-0.660533,-1.241783,-0.040319,-1.579106,1.562825,-0.894687,1.114843,0.891989
3,1000016,school,Alesso,Heroes (we could be),210012,Forever,6WQLkih8nE0JdUCEyLaGnQ,0.163519,-0.810939,-0.626563,0.125442,0.868356,0.476394,-0.066965,-0.304414,0.348512,-0.315390,-0.444021,-0.326381,-0.268484,-0.216030,-0.353737,-0.286299,0.497774,-0.093764,0.218486,0.275453,-0.346192,-0.060127,-0.374081,-0.169846,2.246746,-0.829022,-0.570478,-0.650149,0.243309,-0.041286,0.530145,0.487316,0.386235,-0.098008,-0.099762,-0.115788,0.122960,-0.381961,-0.265588,0.836120,1.040895,0.514868,0.427347,0.726865,2.005331,-0.216795,-0.426145,0.2

#3.4 Creación de DF para playlist
## 📚 Columnas por playlist

| Tipo de variable | Agregación típica | Ejemplo de nombre final |
|------------------|-------------------|-------------------------|
| Descripción playlist | - | `name`, `pid`
| Métricas de audio | media | `avg_bpm`, `avg_energy`, `avg_danceability_ll`, `avg_loudness` |
| Duración total | suma | `total_duration_ms` |
| Probabilidades de género/mood | media | `mean_p_ambient`, `mean_prob_sad`, … |
| Conteo de pistas | n.º de canciones | `n_tracks` |
| Métricas de diversidad (opc.) | entropía | `genre_entropy`, `mood_entropy` |

> **Total aproximado:** ~ 65‑70 columnas por playlist (mismas probabilidades promediadas + 4 promedios de audio + duración + conteo + entropías opcionales).

In [14]:
df_processed_no_metadata = df_processed.copy()
df_processed_no_metadata.drop(columns=['album_name', 'artist_name', 'track_name', 'track_id'], inplace=True)
df_processed_no_metadata.head()

,pid,name,duration_ms,bpm,energy,danceability_ll,loudness,danceability_danceable,gender_female,genre_dortmund_alternative,genre_dortmund_blues,genre_dortmund_electronic,genre_dortmund_folkcountry,genre_dortmund_funksoulrnb,genre_dortmund_jazz,genre_dortmund_pop,genre_dortmund_raphiphop,genre_dortmund_rock,genre_electronic_ambient,genre_electronic_dnb,genre_electronic_house,genre_electronic_techno,genre_electronic_trance,genre_rosamerica_cla,genre_rosamerica_dan,genre_rosamerica_hip,genre_rosamerica_jaz,genre_rosamerica_pop,genre_rosamerica_rhy,genre_rosamerica_roc,genre_rosamerica_spe,genre_tzanetakis_blu,genre_tzanetakis_cla,genre_tzanetakis_cou,genre_tzanetakis_dis,genre_tzanetakis_hip,genre_tzanetakis_jaz,genre_tzanetakis_met,genre_tzanetakis_pop,genre_tzanetakis_reg,genre_tzanetakis_roc,ismir04_rhythm_ChaChaCha,ismir04_rhythm_Jive,ismir04_rhythm_Quickstep,ismir04_rhythm_Rumba-American,ismir04_rhythm_Rumba-International,ismir04_rhythm_Rumba-Misc,ismir04_rhythm_Samba,ismir04_rhythm_Tango,ismir04_rhythm_VienneseWaltz,ismir04_rhythm_Waltz,mood_acoustic_acoustic,mood_aggressive_aggressive,mood_electronic_electronic,mood_happy_happy,mood_party_party,mood_relaxed_relaxed,mood_sad_sad,moods_mirex_Cluster1,moods_mirex_Cluster2,moods_mirex_Cluster3,moods_mirex_Cluster4,moods_mirex_Cluster5,timbre_bright,tonal_atonal_atonal,voice_instrumental_voice
0,1000000,Party,163809,0.160221,0.674865,-0.575001,-0.010197,0.712000,0.890641,-0.498757,-0.430456,0.519981,-0.481145,-0.433349,-0.127378,-0.436943,-0.045659,-0.443713,0.879014,-0.356821,0.216197,0.166095,-0.988342,-0.187154,0.366962,0.586453,-0.120611,-0.472561,0.371606,-0.618070,1.112369,-0.095358,3.040138,-0.406131,1.546956,-1.530079,-1.287515,2.352964,5.155002,2.426936,-0.335286,-0.888938,-0.840955,-0.875683,-0.428931,-0.389895,-0.568154,-0.426928,3.116291,-0.611418,-0.549471,-0.581131,0.685640,1.028789,-1.346029,0.786925,1.130548,0.525529,0.893153,0.420956,-0.730197,1.004184,-0.387829,-1.441935,1.124915,0.680423
1,1000000,Party,166848,0.245847,0.221486,-1.317574,-0.183550,-0.488870,-0.333640,-0.679784,-0.500463,0.647566,-0.520152,-0.540629,-0.312999,-0.539862,-0.219262,-0.502418,-1.123346,-0.471502,3.101279,0.397027,-1.088660,-0.209497,0.602960,1.148218,0.543977,-0.904793,-0.155714,-0.607289,-0.184139,-1.338528,2.918893,-1.099537,-0.730432,-2.426967,-1.133561,1.148927,1.597147,1.152033,4.106680,-0.854069,-0.646158,-0.930438,-0.428940,-0.652143,-0.990493,-0.774483,3.517509,-0.847052,-0.596331,-0.817869,-0.660169,1.069335,-1.418720,-1.359290,0.761099,-0.247641,0.109339,0.095873,0.124757,1.942266,-0.973718,0.130479,0.917806,-1.320410
2,1000000,Party,193058,-0.881112,0.292530,-0.208342,0.569353,0.866053,0.556323,-0.275158,-0.152407,0.138928,-0.043543,-0.158424,0.319389,0.504042,-0.026728,-0.214233,-0.500124,2.074336,1.217905,0.973788,-0.768241,-0.278889,0.024549,-0.043327,-0.287384,-0.125290,1.037323,-0.579063,0.483531,0.202774,-0.064239,0.489081,-0.282863,0.344680,-0.119944,-0.131801,-0.142258,0.081347,0.114261,1.697582,-0.942994,-0.948384,-0.415205,-0.215368,-0.872695,-0.818559,-0.819740,-0.763915,-0.599103,-0.767869,2.182934,0.428434,-1.265545,1.156226,-0.448820,-0.589449,-0.660533,-1.241783,-0.040319,-1.579106,1.562825,-0.894687,1.114843,0.891989
3,1000016,school,210012,0.163519,-0.810939,-0.626563,0.125442,0.868356,0.476394,-0.066965,-0.304414,0.348512,-0.315390,-0.444021,-0.326381,-0.268484,-0.216030,-0.353737,-0.286299,0.497774,-0.093764,0.218486,0.275453,-0.346192,-0.060127,-0.374081,-0.169846,2.246746,-0.829022,-0.570478,-0.650149,0.243309,-0.041286,0.530145,0.487316,0.386235,-0.098008,-0.099762,-0.115788,0.122960,-0.381961,-0.265588,0.836120,1.040895,0.514868,0.427347,0.726865,2.005331,-0.216795,-0.426145,0.256126,-0.796063,-0.660169,1.027652,0.961514,-1.361806,0.761068,0.333386,0.225041,0.561850,-0.327078,0.033352,-0.169638,0.538302,0.921453,0.575065
4,1000016,school,194240,-0.875119,-0.751482,0.471480,0.127868,0.619398,-1.057473,-0.549062,-0.480943,0.585538,-0.424023,-0.557668

In [15]:
from scipy.stats import entropy
# 1)  —  Qué columnas numéricas promediar  —
basic_avg    = ["bpm", "energy", "danceability_ll", "loudness"]
duration_col = "duration_ms"

# todas las columnas numéricas que NO son IDs ni duración
num_cols = (
    df_processed.select_dtypes("number")
    .columns.difference(["pid", "track_id", duration_col] + basic_avg)
)

# 2)  —  Diccionario de agregaciones por playlist —
agg_dict = {c: "mean" for c in basic_avg}      # métricas de audio
agg_dict.update({c: "mean" for c in num_cols}) # probabilidades género/mood
agg_dict[duration_col] = "sum"                 # duración total

df_playlist = (
    df_processed
      .groupby("pid")
      .agg(agg_dict)
      .rename(columns={
          "bpm": "avg_bpm",
          "energy": "avg_energy",
          "danceability_ll": "avg_danceability_ll",
          "loudness": "avg_loudness",
          duration_col: "total_duration_ms"})
      .reset_index()
)

# 3)  —  Número de canciones por playlist —
df_playlist["n_tracks"] = (
    df_processed.groupby("pid")["track_id"].nunique().values
)


# columnas de género (todas las que empiezan con los prefijos indicados)
genre_prefixes = (
    "genre_dortmund_", "genre_electronic_", "genre_rosamerica_",
    "genre_tzanetakis_", "ismir04_rhythm_"
)
genre_cols = [c for c in df_playlist.columns if c.startswith(genre_prefixes)]

mood_prefixes = ("mood_",)   # p.ej. mood_happy_happy, mood_sad_sad, …
mood_cols  = [c for c in df_playlist.columns if c.startswith(mood_prefixes)]



df_playlist.head()

,pid,avg_bpm,avg_energy,avg_danceability_ll,avg_loudness,danceability_danceable,gender_female,genre_dortmund_alternative,genre_dortmund_blues,genre_dortmund_electronic,genre_dortmund_folkcountry,genre_dortmund_funksoulrnb,genre_dortmund_jazz,genre_dortmund_pop,genre_dortmund_raphiphop,genre_dortmund_rock,genre_electronic_ambient,genre_electronic_dnb,genre_electronic_house,genre_electronic_techno,genre_electronic_trance,genre_rosamerica_cla,genre_rosamerica_dan,genre_rosamerica_hip,genre_rosamerica_jaz,genre_rosamerica_pop,genre_rosamerica_rhy,genre_rosamerica_roc,genre_rosamerica_spe,genre_tzanetakis_blu,genre_tzanetakis_cla,genre_tzanetakis_cou,genre_tzanetakis_dis,genre_tzanetakis_hip,genre_tzanetakis_jaz,genre_tzanetakis_met,genre_tzanetakis_pop,genre_tzanetakis_reg,genre_tzanetakis_roc,ismir04_rhythm_ChaChaCha,ismir04_rhythm_Jive,ismir04_rhythm_Quickstep,ismir04_rhythm_Rumba-American,ismir04_rhythm_Rumba-International,ismir04_rhythm_Rumba-Misc,ismir04_rhythm_Samba,ismir04_rhythm_Tango,ismir04_rhythm_VienneseWaltz,ismir04_rhythm_Waltz,mood_acoustic_acoustic,mood_aggressive_aggressive,mood_electronic_electronic,mood_happy_happy,mood_party_party,mood_relaxed_relaxed,mood_sad_sad,moods_mirex_Cluster1,moods_mirex_Cluster2,moods_mirex_Cluster3,moods_mirex_Cluster4,moods_mirex_Cluster5,timbre_bright,tonal_atonal_atonal,voice_instrumental_voice,total_duration_ms,n_tracks
0,1000000,-0.158348,0.396294,-0.700306,0.125202,0.363061,0.371108,-0.484566,-0.361109,0.435491,-0.348280,-0.377467,-0.040329,-0.157588,-0.097216,-0.386788,-0.248152,0.415338,1.511793,0.512304,-0.948414,-0.225180,0.331491,0.563781,0.045327,-0.500881,0.417738,-0.601474,0.470587,-0.410371,1.964931,-0.338862,0.177887,-1.204122,-0.847007,1.123363,2.203297,1.220105,1.295219,-0.015142,-0.810036,-0.918169,-0.424359,-0.419135,-0.810447,-0.673323,1.938020,-0.740795,-0.581635,-0.722290,0.736135,0.842186,-1.343432,0.194620,0.480942,-0.103854,0.113987,-0.241651,-0.215253,0.455781,0.067093,-0.735381,1.052521,0.084001,523715,3
1,1000001,-0.512764,0.251691,0.040809,-0.416864,-0.915044,0.089547,-0.372681,-0.116873,0.381763,-0.352400,-0.216128,-0.247667,-0.311930,-0.202284,-0.261509,0.519464,-0.639666,-0.212279,-0.132568,-0.249833,0.019684,-0.413942,-0.431987,1.194941,-0.353920,0.995258,-0.344037,1.021559,0.315765,0.066730,0.068895,0.187204,0.167479,-0.280955,0.105875,0.222843,0.230608,0.381979,-0.285157,-0.247098,-0.391784,0.794242,-0.309223,-0.132248,-0.098008,-0.227481,0.469300,-0.308003,0.406892,-0.078806,0.166418,-0.300251,-0.396308,0.190350,0.611128,-0.155772,-0.609458,0.215563,-0.632964,0.527874,0.203971,0.131274,-0.285914,3294637,10
2,1000009,0.417665,0.803308,-0.719781,-0.689314,-0.952088,-0.402470,0.134985,1.360055,-0.896558,1.042715,0.451415,0.772938,0.226602,0.077962,0.502927,0.829002,-0.662246,-0.324779,-0.110551,-0.473228,-0.087438,-0.390645,-0.525870,0.468216,0.120518,1.026552,-0.422250,1.236388,-0.293381,-0.066736,0.270279,0.152632,-0.425808,-0.236037,-0.311570,-0.069070,0.050828,1.091356,-0.737798,-0.346437,0.027060,-0.000244,0.601823,1.405810,-0.352271,0.278333,0.609312,0.495119,1.603482,-0.451481,-1.089573,0.101270,-0.870960,1.190770,1.353686,-0.124238,0.484062,0.950558,0.240725,-0.944949,0.086277,-0.089870,0.502085,948799,5
3,1000016,-0.502793,-0.661331,-0.120105,0.180342,0.682097,-0.612838,-0.260205,-0.319518,0.375747,-0.255350,-0.403510,-0.307718,-0.362986,-0.216109,-0.322164,-0.020339,0.186082,0.434574,0.008449,-0.310946,-0.140528,-0.190741,-0.234348,-0.056728,0.965302,-0.440888,-0.005007,-0.016832,0.199535,-0.065891,-0.045209,0.208509,0.342959,-0.120767,0.031221,0.060558,-0.134902,0.101926,-0.443774,0.758718,1.168188,0.083235,0.078435,0.383842,0.861546,0.762453,-0.477459,0.050534,-0.508970,0.046480,-0.187496,-0.339649,-0.129982,0.171618,-0.053440,0.374590,0.988825,-0.395497,0.626471,-0.656781,0.861908,-0.326220,0.386508,594092,3
4,1000020,-1.077283,-0.688983,0.546987,0.640798,0.783371,-1.097435,0.774962,0.549553,-0.859564,0.816959,1.726948,0.246975,0.555984,

In [18]:
# Añadiendo el nombre otra vez
df_playlist = df_playlist.merge(df_processed[['pid', 'name']], on='pid', how='left')
df_playlist = df_playlist.drop_duplicates(subset=['pid']) # Ensure unique playlists
print(df_playlist.head())

        pid   avg_bpm  avg_energy  avg_danceability_ll  avg_loudness  \
0   1000000 -0.158348    0.396294            -0.700306      0.125202   
3   1000001 -0.512764    0.251691             0.040809     -0.416864   
14  1000009  0.417665    0.803308            -0.719781     -0.689314   
19  1000016 -0.502793   -0.661331            -0.120105      0.180342   
22  1000020 -1.077283   -0.688983             0.546987      0.640798   

    danceability_danceable  gender_female  genre_dortmund_alternative  \
0                 0.363061       0.371108                   -0.484566   
3                -0.915044       0.089547                   -0.372681   
14               -0.952088      -0.402470                    0.134985   
19                0.682097      -0.612838                   -0.260205   
22                0.783371      -1.097435                    0.774962   

    genre_dortmund_blues  genre_dortmund_electronic  \
0              -0.361109                   0.435491   
3              -0.

In [17]:
df_playlist.head()

,pid,avg_bpm,avg_energy,avg_danceability_ll,avg_loudness,danceability_danceable,gender_female,genre_dortmund_alternative,genre_dortmund_blues,genre_dortmund_electronic,genre_dortmund_folkcountry,genre_dortmund_funksoulrnb,genre_dortmund_jazz,genre_dortmund_pop,genre_dortmund_raphiphop,genre_dortmund_rock,genre_electronic_ambient,genre_electronic_dnb,genre_electronic_house,genre_electronic_techno,genre_electronic_trance,genre_rosamerica_cla,genre_rosamerica_dan,genre_rosamerica_hip,genre_rosamerica_jaz,genre_rosamerica_pop,genre_rosamerica_rhy,genre_rosamerica_roc,genre_rosamerica_spe,genre_tzanetakis_blu,genre_tzanetakis_cla,genre_tzanetakis_cou,genre_tzanetakis_dis,genre_tzanetakis_hip,genre_tzanetakis_jaz,genre_tzanetakis_met,genre_tzanetakis_pop,genre_tzanetakis_reg,genre_tzanetakis_roc,ismir04_rhythm_ChaChaCha,ismir04_rhythm_Jive,ismir04_rhythm_Quickstep,ismir04_rhythm_Rumba-American,ismir04_rhythm_Rumba-International,ismir04_rhythm_Rumba-Misc,ismir04_rhythm_Samba,ismir04_rhythm_Tango,ismir04_rhythm_VienneseWaltz,ismir04_rhythm_Waltz,mood_acoustic_acoustic,mood_aggressive_aggressive,mood_electronic_electronic,mood_happy_happy,mood_party_party,mood_relaxed_relaxed,mood_sad_sad,moods_mirex_Cluster1,moods_mirex_Cluster2,moods_mirex_Cluster3,moods_mirex_Cluster4,moods_mirex_Cluster5,timbre_bright,tonal_atonal_atonal,voice_instrumental_voice,total_duration_ms,n_tracks,name
0,1000000,-0.158348,0.396294,-0.700306,0.125202,0.363061,0.371108,-0.484566,-0.361109,0.435491,-0.348280,-0.377467,-0.040329,-0.157588,-0.097216,-0.386788,-0.248152,0.415338,1.511793,0.512304,-0.948414,-0.225180,0.331491,0.563781,0.045327,-0.500881,0.417738,-0.601474,0.470587,-0.410371,1.964931,-0.338862,0.177887,-1.204122,-0.847007,1.123363,2.203297,1.220105,1.295219,-0.015142,-0.810036,-0.918169,-0.424359,-0.419135,-0.810447,-0.673323,1.938020,-0.740795,-0.581635,-0.722290,0.736135,0.842186,-1.343432,0.194620,0.480942,-0.103854,0.113987,-0.241651,-0.215253,0.455781,0.067093,-0.735381,1.052521,0.084001,523715,3,Party
3,1000001,-0.512764,0.251691,0.040809,-0.416864,-0.915044,0.089547,-0.372681,-0.116873,0.381763,-0.352400,-0.216128,-0.247667,-0.311930,-0.202284,-0.261509,0.519464,-0.639666,-0.212279,-0.132568,-0.249833,0.019684,-0.413942,-0.431987,1.194941,-0.353920,0.995258,-0.344037,1.021559,0.315765,0.066730,0.068895,0.187204,0.167479,-0.280955,0.105875,0.222843,0.230608,0.381979,-0.285157,-0.247098,-0.391784,0.794242,-0.309223,-0.132248,-0.098008,-0.227481,0.469300,-0.308003,0.406892,-0.078806,0.166418,-0.300251,-0.396308,0.190350,0.611128,-0.155772,-0.609458,0.215563,-0.632964,0.527874,0.203971,0.131274,-0.285914,3294637,10,Oldies
14,1000009,0.417665,0.803308,-0.719781,-0.689314,-0.952088,-0.402470,0.134985,1.360055,-0.896558,1.042715,0.451415,0.772938,0.226602,0.077962,0.502927,0.829002,-0.662246,-0.324779,-0.110551,-0.473228,-0.087438,-0.390645,-0.525870,0.468216,0.120518,1.026552,-0.422250,1.236388,-0.293381,-0.066736,0.270279,0.152632,-0.425808,-0.236037,-0.311570,-0.069070,0.050828,1.091356,-0.737798,-0.346437,0.027060,-0.000244,0.601823,1.405810,-0.352271,0.278333,0.609312,0.495119,1.603482,-0.451481,-1.089573,0.101270,-0.870960,1.190770,1.353686,-0.124238,0.484062,0.950558,0.240725,-0.944949,0.086277,-0.089870,0.502085,948799,5,Study Music
19,1000016,-0.502793,-0.661331,-0.120105,0.180342,0.682097,-0.612838,-0.260205,-0.319518,0.375747,-0.255350,-0.403510,-0.307718,-0.362986,-0.216109,-0.322164,-0.020339,0.186082,0.434574,0.008449,-0.310946,-0.140528,-0.190741,-0.234348,-0.056728,0.965302,-0.440888,-0.005007,-0.016832,0.199535,-0.065891,-0.045209,0.208509,0.342959,-0.120767,0.031221,0.060558,-0.134902,0.101926,-0.443774,0.758718,1.168188,0.083235,0.078435,0.383842,0.861546,0.762453,-0.477459,0.050534,-0.508970,0.046480,-0.187496,-0.339649,-0.129982,0.171618,-0.053440,0.374590,0.988825,-0.395497,0.626471,-0.656781,0.861908,-0.326220,0.386508,594092,3,school
22,1000020,-1.077283,-0.688983,0.546987,0.640798,0.783371,-1.097435,0.774962,0.549553,-0.859

In [ ]:
df_playlist.describe()


,pid,avg_bpm,avg_energy,avg_danceability_ll,avg_loudness,danceability_danceable,gender_female,genre_dortmund_alternative,genre_dortmund_blues,genre_dortmund_electronic,genre_dortmund_folkcountry,genre_dortmund_funksoulrnb,genre_dortmund_jazz,genre_dortmund_pop,genre_dortmund_raphiphop,genre_dortmund_rock,genre_electronic_ambient,genre_electronic_dnb,genre_electronic_house,genre_electronic_techno,genre_electronic_trance,genre_rosamerica_cla,genre_rosamerica_dan,genre_rosamerica_hip,genre_rosamerica_jaz,genre_rosamerica_pop,genre_rosamerica_rhy,genre_rosamerica_roc,genre_rosamerica_spe,genre_tzanetakis_blu,genre_tzanetakis_cla,genre_tzanetakis_cou,genre_tzanetakis_dis,genre_tzanetakis_hip,genre_tzanetakis_jaz,genre_tzanetakis_met,genre_tzanetakis_pop,genre_tzanetakis_reg,genre_tzanetakis_roc,ismir04_rhythm_ChaChaCha,ismir04_rhythm_Jive,ismir04_rhythm_Quickstep,ismir04_rhythm_Rumba-American,ismir04_rhythm_Rumba-International,ismir04_rhythm_Rumba-Misc,ismir04_rhythm_Samba,ismir04_rhythm_Tango,ismir04_rhythm_VienneseWaltz,ismir04_rhythm_Waltz,mood_acoustic_acoustic,mood_aggressive_aggressive,mood_electronic_electronic,mood_happy_happy,mood_party_party,mood_relaxed_relaxed,mood_sad_sad,moods_mirex_Cluster1,moods_mirex_Cluster2,moods_mirex_Cluster3,moods_mirex_Cluster4,moods_mirex_Cluster5,timbre_bright,tonal_atonal_atonal,voice_instrumental_voice,total_duration_ms,n_tracks
count,8.192000e+03,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8.192000e+03,8192.000000
mean,1.013236e+06,-0.030277,0.030638,0.040049,-0.038286,-0.002645,0.078626,-0.055435,-0.023060,0.040536,-0.023674,-0.039928,-0.003630,-0.043126,-0.004354,-0.053378,-0.007159,-0.031125,0.034142,0.013593,-0.014557,0.019830,0.038738,0.027527,0.043140,-0.026852,0.047427,-0.094066,0.029932,-0.006392,0.010141,-0.032455,-0.046891,-0.024625,-0.001890,0.012260,0.018135,-0.000845,0.036965,0.028735,-0.059890,-0.004529,0.006606,-0.005607,0.010212,0.000588,0.019202,-0.038787,0.006917,0.014712,-0.012548,0.037713,-0.045478,-0.023414,0.057503,0.032535,-0.025892,-0.024356,0.002153,-0.010844,0.022636,-0.012357,0.044119,-0.035646,3.480649e+06,15.027222
std,1.225240e+04,0.529898,0.634009,0.685440,0.657958,0.703541,0.621229,0.597077,0.648349,0.608993,0.663666,0.554068,0.576279,0.561812,0.538956,0.552437,0.701771,0.558564,0.662205,0.636499,0.657007,0.667702,0.656851,0.719712,0.716058,0.633424,0.652610,0.675700,0.622904,0.562986,0.580638,0.569689,0.562325,0.573257,0.546916,0.574371,0.573169,0.555220,0.570936,0.646406,0.568211,0.559290,0.571399,0.567431,0.607665,0.590627,0.593934,0.633045,0.607282,0.695506,0.630059,0.722065,0.604860,0.638978,0.607943,0.670425,0.551014,0.598152,0.610405,0.598831,0.645564,0.647788,0.647073,0.588660,4.164845e+06,18.035319
min,1.000000e+06,-1.780560,-1.759236,-1.704166,-4.075725,-1.807466,-1.496365,-0.697431,-0.530290,-3.035332,-0.527833,-0.584842,-0.533139,-0.548396,-0.234339,-0.518965,-1.593131,-0.936827,-1.122629,-1.033024,-1.231250,-0.356121,-0.551187,-0.632662,-0.484082,-1.122733,-1.009003,-0.647764,-0.723965,-4.031121,-2.377616,-3.587805,-3.210672,-3.561200,-1.456010,-3.155016,-2.645039,-3.866258,-1.935275,-1.064498,-1.095456,-1.237550,-0.546040,-0.758802,-1.082905,-1.161460,-0.857400,-0.879940,-0.681366,-0.817890,-0.660169,-1.738417,-1.456123,-1.361957,-1.726071,-1.5

# 3.7 Reducción de Dimensionalidad (PCA y UMAP)
Finalmente, aplicamos PCA para reducir la dimensionalidad de las features numéricas, útil para el clustering, y UMAP para visualización.

## para **TRACKS**

In [19]:
# --- 3.7 Reducción de dimensionalidad ---

# Usaremos el DataFrame de tracks únicos con las features ya imputadas y escaladas
# Primero, preparamos tracks_df_unique con los datos procesados
tracks_df_unique_processed = df_processed.drop_duplicates(subset=['track_id']).set_index('track_id')


# --- PCA ---
pca = PCA(n_components=10) # Reducimos a 10 componentes como ejemplo
track_features_pca = pca.fit_transform(tracks_df_unique_processed[numerical_cols])

print(f"Varianza explicada por los {pca.n_components_} componentes: {np.sum(pca.explained_variance_ratio_):.2f}")

# Crear un DataFrame con los componentes principales
df_pca = pd.DataFrame(track_features_pca, index=tracks_df_unique_processed.index, columns=[f'PC_{i+1}' for i in range(pca.n_components_)])
print("\nDataFrame con Componentes Principales (PCA):")
df_pca.head()


# --- UMAP (para visualización 2D) ---
reducer_umap = umap.UMAP(n_components=2, random_state=42)
embedding = reducer_umap.fit_transform(tracks_df_unique_processed[numerical_cols])

# Crear un DataFrame con las dimensiones UMAP
df_umap = pd.DataFrame(embedding, index=tracks_df_unique_processed.index, columns=['UMAP_1', 'UMAP_2'])
print("\nDataFrame con componentes UMAP para visualización:")
df_umap.head()


Varianza explicada por los 10 componentes: 0.66

DataFrame con Componentes Principales (PCA):


/usr/local/lib/python3.11/dist-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(



DataFrame con componentes UMAP para visualización:


,UMAP_1,UMAP_2
track_id,,
66U0ASk1VHZsqIkpMjKX3B,9.075862,6.171213
5MhsZlmKJG6X5kTHkdwC4B,9.056919,6.946337
3G6hD9B2ZHOsgf4WfNu7X1,14.568500,7.935036
6WQLkih8nE0JdUCEyLaGnQ,10.718304,10.415011
5j9iuo3tMmQIfnEEQOOjxh,9.530182,10.419980


para **PLAYLIST**

In [20]:
# --- 3.7 Reducción de dimensionalidad ---

# Usaremos el DataFrame de playlist únicos con las features ya imputadas y escaladas
# Primero, preparamos playlist_df_unique con los datos procesados
playlist_df_unique_processed = df_processed.drop_duplicates(subset=['pid']).set_index('pid')


# --- PCA ---
pca = PCA(n_components=10) # Reducimos a 10 componentes como ejemplo
playlist_features_pca = pca.fit_transform(playlist_df_unique_processed[numerical_cols])

print(f"Varianza explicada por los {pca.n_components_} componentes: {np.sum(pca.explained_variance_ratio_):.2f}")

# Crear un DataFrame con los componentes principales
playlist_pca = pd.DataFrame(playlist_features_pca, index=playlist_df_unique_processed.index, columns=[f'PC_{i+1}' for i in range(pca.n_components_)])
print("\nDataFrame con Componentes Principales (PCA):")
playlist_pca.head()



Varianza explicada por los 10 componentes: 0.65

DataFrame con Componentes Principales (PCA):


,PC_1,PC_2,PC_3,PC_4,PC_5,PC_6,PC_7,PC_8,PC_9,PC_10
pid,,,,,,,,,,
1000000,-0.663803,-0.631957,4.806233,2.348698,-1.628967,-0.781611,3.238690,1.171119,-2.040517,2.240854
1000016,-1.349871,0.000669,0.607624,-0.475811,1.879978,2.161518,-2.195647,0.174255,0.376852,0.624177
1000020,0.233109,4.628954,-0.889581,0.257637,-0.817259,0.401576,-0.180352,0.579727,0.363847,0.005663
1000023,-3.502215,0.524163,-0.833910,-0.632196,-1.481683,0.398017,-0.946594,4.065633,-1.825655,-1.087756
1000040,-0.979415,-1.463812,0.872277,0.829315,0.013625,2.956005,-0.203531,-0.288747,-0.247835,0.576816


In [21]:
playlist_pca.shape

(8192, 10)

In [22]:

# --- UMAP (para visualización 2D) ---
reducer_umap = umap.UMAP(n_components=2, random_state=42)
embedding = reducer_umap.fit_transform(playlist_df_unique_processed[numerical_cols])

# Crear un DataFrame con las dimensiones UMAP
playlist_umap = pd.DataFrame(embedding, index=playlist_df_unique_processed.index, columns=['UMAP_1', 'UMAP_2'])
print("\nDataFrame con componentes UMAP para visualización:")
playlist_umap.head()

/usr/local/lib/python3.11/dist-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(



DataFrame con componentes UMAP para visualización:


,UMAP_1,UMAP_2
pid,,
1000000,17.085173,-0.839397
1000016,6.023946,3.598349
1000020,8.524440,1.867325
1000023,9.937390,6.673456
1000040,4.498182,0.070753


In [23]:
#Guardar dataframes para Reglas de Asociacion y posteriormente para clustering

import pandas as pd
from pathlib import Path

# --- Crear un directorio para los datos procesados ---
output_dir = Path("data/processed_for_modeling")
output_dir.mkdir(parents=True, exist_ok=True)

print(f"Directorio de salida creado en: {output_dir}")

# --- Guardar los DataFrames esenciales ---

# 1. El DataFrame principal, totalmente procesado
# Útil para la agregación de playlists o futuras exploraciones.
df_processed.to_parquet(output_dir / "df_processed_full.parquet")
print("1. 'df_processed_full.parquet' guardado.")


# 3. El DataFrame para Clustering de Tracks
# Contiene las features únicas por canción, ya escaladas.
tracks_df_unique_processed.to_parquet(output_dir / "tracks_for_clustering.parquet")
print("3. 'tracks_for_clustering.parquet' guardado.")

# 4. (Opcional pero recomendado) Guardar los datos con dimensionalidad reducida
if 'df_pca' in locals():
    df_pca.to_parquet(output_dir / "df_pca_components.parquet")
    print("4. 'df_pca_components.parquet' guardado.")

if 'df_umap' in locals():
    df_umap.to_parquet(output_dir / "df_umap_embedding.parquet")
    print("5. 'df_umap_embedding.parquet' guardado.")

print("\n¡Todos los archivos han sido guardados exitosamente!")
print("Puedes cargarlos en tu próximo notebook con pd.read_parquet()")

Directorio de salida creado en: data/processed_for_modeling
1. 'df_processed_full.parquet' guardado.
3. 'tracks_for_clustering.parquet' guardado.
4. 'df_pca_components.parquet' guardado.
5. 'df_umap_embedding.parquet' guardado.

¡Todos los archivos han sido guardados exitosamente!
Puedes cargarlos en tu próximo notebook con pd.read_parquet()


In [24]:
# --- Crear un directorio para los datos procesados ---
output_dir = Path("data/processed_for_modeling")
output_dir.mkdir(parents=True, exist_ok=True)

print(f"Directorio de salida creado en: {output_dir}")

# --- Guardar los DataFrames esenciales ---

# 1. El DataFrame principal, totalmente procesado
# Útil para la agregación de playlists o futuras exploraciones.
df_playlist.to_parquet(output_dir / "df_playlist_full.parquet")
print("1. 'playlist_processed_full.parquet' guardado.")


# 3. El DataFrame para Clustering de Tracks
# Contiene las features únicas por canción, ya escaladas.
playlist_df_unique_processed.to_parquet(output_dir / "playlist_for_clustering.parquet")
print("3. 'tracks_for_clustering.parquet' guardado.")

# 4. (Opcional pero recomendado) Guardar los datos con dimensionalidad reducida
if 'playlist_pca' in locals():
    playlist_pca.to_parquet(output_dir / "playlist_pca_components.parquet")
    print("4. 'playlist_pca_components.parquet' guardado.")

if 'playlist_umap' in locals():
    playlist_umap.to_parquet(output_dir / "playlist_umap_embedding.parquet")
    print("5. 'playlist_umap_embedding.parquet' guardado.")

print("\n¡Todos los archivos han sido guardados exitosamente!")
print("Puedes cargarlos en tu próximo notebook con pd.read_parquet()")

Directorio de salida creado en: data/processed_for_modeling
1. 'playlist_processed_full.parquet' guardado.
3. 'tracks_for_clustering.parquet' guardado.
4. 'playlist_pca_components.parquet' guardado.
5. 'playlist_umap_embedding.parquet' guardado.

¡Todos los archivos han sido guardados exitosamente!
Puedes cargarlos en tu próximo notebook con pd.read_parquet()


# ❌ código no usado en este taller 3.5 Discretizar para Asociación (KBinsDiscretizer)
Este código se reutilizará en el próximo taller porque se hará asociación dentro del clustering

In [ ]:
# --- 3.5 Discretizar para Asociación ---

# Seleccionamos las columnas a discretizar
cols_to_discretize = ['energy', 'danceability_ll', 'bpm']

# Crear el discretizador
discretizer = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='uniform', subsample=None)

# Aplicar y crear nuevas columnas
for col in cols_to_discretize:
    # Creamos una nueva columna para no sobreescribir la escalada
    new_col_name = f'{col}_cat'
    df_processed[new_col_name] = discretizer.fit_transform(df_processed[[col]])
    # Mapear los números a etiquetas claras
    df_processed[new_col_name] = df_processed[new_col_name].map({0.0: 'bajo', 1.0: 'medio', 2.0: 'alto'})

print("Nuevas columnas discretizadas:")
df_processed[['energy_cat', 'danceability_ll_cat', 'bpm_cat']].head()



## 3.6 Binarizar Pista-Playlist (Matriz de Transacciones)
Esta es la preparación final para los algoritmos de reglas de asociación (Apriori, FP-Growth). Crearemos una matriz donde cada fila es una playlist (pid) y las columnas son las canciones (track_uri), con un 1 si la canción está en la playlist.

Nota: Esta df_transactions es el input directo para mlxtend.frequent_patterns.



In [ ]:
!pip install --extra-index-url=https://pypi.nvidia.com cudf-cu11 cupy-cuda11x

In [ ]:
# --- 3.6 Binarizar Pista-Playlist ---

# Usaremos pd.crosstab para crear la matriz de transacciones de manera eficiente
# Usamos un subconjunto para el ejemplo, ya que la matriz completa puede ser muy grande
# Aquí se recomienda usar los track_id en lugar de track_uri si son más limpios
# df_transactions = pd.crosstab(df_processed['pid'], df_processed['track_id'])


def create_transaction_matrix_optimized(df_processed, chunk_size=50000):
    """
    Versión optimizada usando pandas con procesamiento por chunks
    Más eficiente en memoria y funciona sin GPU
    """
    print("Creando matriz de transacciones de forma optimizada...")

    # Usar solo las columnas necesarias
    df_subset = df_processed[['pid', 'track_id']].copy()

    # Eliminar duplicados si existen
    df_subset = df_subset.drop_duplicates()

    print(f"Datos únicos: {len(df_subset)} filas")
    print(f"Playlists únicas: {df_subset['pid'].nunique()}")
    print(f"Tracks únicos: {df_subset['track_id'].nunique()}")

    # Crear la matriz de transacciones
    df_transactions = pd.crosstab(
        df_subset['pid'],
        df_subset['track_id'],
        dropna=False
    )

    # Convertir a binario y optimizar tipo de datos
    df_transactions = (df_transactions > 0).astype('int8')

    print(f"Matriz creada. Shape: {df_transactions.shape}")
    print(f"Memoria utilizada: {df_transactions.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

    return df_transactions


df_transactions = create_transaction_matrix_optimized(df_processed)
df_transactions.head()

In [ ]:
#guardar en disco el df_transactions
# 2. La matriz de transacciones para Reglas de Asociación
# La guardamos como Parquet, que maneja bien las matrices sparse si se convierten.
# O como CSV si se prefiere mayor compatibilidad.
df_transactions.to_parquet(output_dir / "transactions_matrix.parquet")
print("2. 'transactions_matrix.parquet' guardado.")
